In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm


# Métricas Epidemiológicas

In [ ]:
pathArquivo = '../../data/processed/df_processed.csv'
print(pathArquivo)

In [ ]:
dfMetricas = pd.read_csv(pathArquivo, sep=",", encoding="UTF-8", na_values=["NA"])
print("Colunas disponíveis:", list(dfMetricas))


In [ ]:
colunas = [
    "dataNotificacao", "dataInicioSintomas", "dataEncerramento",
    "estado", "estadoIBGE", "municipio", "municipioIBGE",
    "classificacaoFinal", "evolucaoCaso",
    "codigoResultadoTeste1", "codigoResultadoTeste2", "codigoResultadoTeste3", "codigoResultadoTeste4",
    "dataColetaTeste1", "dataColetaTeste2", "dataColetaTeste3", "dataColetaTeste4",
    "idade", "sexo"
]

dfMetricasEpidemicas = dfMetricas[colunas]


In [ ]:
dfMetricasEpidemicas

## Gerar colunas para identificar Resultado dos casos

#### Resultado dos casos

- 1. Reagente/Detectável 
- 2. Não Reagente/Não detectável 
- 9. Inválido/ Inconclusivo ou Indeterminado

In [ ]:
dfMetricasEpidemicas["casosConfirmados"] = (dfMetricasEpidemicas["codigoResultadoTeste1"] == 1.0).astype(int)
dfMetricasEpidemicas["casosDescartados"] = (dfMetricasEpidemicas["codigoResultadoTeste1"] == 2.0).astype(int)
dfMetricasEpidemicas["casosInconclusivos"] = (dfMetricasEpidemicas["codigoResultadoTeste1"] == 9.0).astype(int)

## Gerar colunas para identificar evolução dos casos

#### Evolução do Caso

 - Em tratamento 
- domiciliar 
- Internado em UTI 
- Internado 
- Óbito 
- Cura 	

In [ ]:

dfMetricasEpidemicas["Óbito"] = (dfMetricasEpidemicas["evolucaoCaso"]=="Óbito").astype(int)
dfMetricasEpidemicas["Cura"] = (dfMetricasEpidemicas["evolucaoCaso"]=="Cura").astype(int)

In [ ]:
dfMetricasEpidemicas["Populacao"] = 1

In [ ]:
dfMetricasEpidemicas["FaixaIdade"] = "sem referencia"
dfMetricasEpidemicas.loc[(dfMetricasEpidemicas["idade"] >= 0) & (dfMetricasEpidemicas["idade"] < 18), "FaixaIdade"] = "0-18"
dfMetricasEpidemicas.loc[(dfMetricasEpidemicas["idade"] >= 18) & (dfMetricasEpidemicas["idade"] < 30), "FaixaIdade"] = "19-30"
dfMetricasEpidemicas.loc[(dfMetricasEpidemicas["idade"] >= 30) & (dfMetricasEpidemicas["idade"] < 45), "FaixaIdade"] = "31-45"
dfMetricasEpidemicas.loc[(dfMetricasEpidemicas["idade"] >= 45) & (dfMetricasEpidemicas["idade"] < 60), "FaixaIdade"] = "46-60"
dfMetricasEpidemicas.loc[(dfMetricasEpidemicas["idade"] >= 60) & (dfMetricasEpidemicas["idade"] < 75), "FaixaIdade"] = "61-75"
dfMetricasEpidemicas.loc[dfMetricasEpidemicas["idade"] >= 75, "FaixaIdade"] = "76+"

In [ ]:
colunas = [
    "dataNotificacao", "dataInicioSintomas", "dataEncerramento","estado", "estadoIBGE", "municipio",
    "classificacaoFinal","dataColetaTeste1","FaixaIdade", "sexo",
    "Óbito","Cura","casosConfirmados","casosDescartados","casosInconclusivos","Populacao"
]

dfFinal = dfMetricasEpidemicas[colunas]


### Utilizando MatplotLib

In [ ]:
total_confirmados = dfFinal["casosConfirmados"].sum()
total_descartados = dfFinal["casosDescartados"].sum()
total_inconclusivos = dfFinal["casosInconclusivos"].sum()

total_testados = total_confirmados + total_descartados
taxa_positividade = (total_confirmados / total_testados) * 100

total_obitos = dfFinal["Óbito"].sum()
taxa_letalidade = (total_obitos / total_confirmados) * 100

total_populacao = dfFinal["Populacao"].sum()
taxa_mortalidade = (total_obitos / total_populacao) * 100000

total_recuperados = dfFinal["Cura"].sum()
taxa_recuperacao = (total_recuperados / total_confirmados) * 100

In [ ]:
# Dados
casos = ["Confirmados", "Descartados", "Inconclusivos"]
valores = [total_confirmados, total_descartados, total_inconclusivos]

# Estilo
plt.style.use('seaborn')

# Configuração do Gráfico
plt.figure(figsize=(10, 6))
cores = cm.get_cmap("Set2", len(valores))
plt.bar(casos, valores, color=[cores(i) for i in range(len(valores))])

# Títulos e Rótulos
plt.title("Distribuição de Casos por Categoria", fontsize=16, fontweight='bold')
plt.xlabel("Categoria", fontsize=12)
plt.ylabel("Total de Casos", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Grade e Rótulos
plt.grid(axis='y', linestyle='--', alpha=0.7)
for i, valor in enumerate(valores):
    plt.text(i, valor + 50, str(valor), ha='center', fontsize=10)

# Salvar e Mostrar
plt.tight_layout()
plt.savefig("distribuicao_casos.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Dados
casos = ["População Geral", "Testes Realizados", "Recuperados", "Óbitos"]
valores = [total_populacao,total_testados, total_recuperados, total_obitos]


# Estilo
plt.style.use('seaborn')

# Configuração do Gráfico
plt.figure(figsize=(10, 6))
cores = cm.get_cmap("Set2", len(valores))
plt.bar(casos, valores, color=[cores(i) for i in range(len(valores))])

# Títulos e Rótulos
plt.title("Distribuição de Casos por Categoria", fontsize=16, fontweight='bold')
plt.xlabel("Categoria", fontsize=12)
plt.ylabel("Total de Casos", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Grade e Rótulos
plt.grid(axis='y', linestyle='--', alpha=0.7)
for i, valor in enumerate(valores):
    plt.text(i, valor + 50, str(valor), ha='center', fontsize=10)

# Salvar e Mostrar
plt.tight_layout()
plt.savefig("Mapeamento_Casos.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:

# Dados de exemplo (substitua pelos seus valores)
masculino_total_testados = 7000
feminino_total_testados = 8000
masculino_confirmados = 3500
feminino_confirmados = 4000

# Cálculos
masculino_positividade = (masculino_confirmados / masculino_total_testados) * 100
feminino_positividade = (feminino_confirmados / feminino_total_testados) * 100

# Dados para o gráfico
labels = ["Masculino", "Feminino"]
total_testados = [masculino_total_testados, feminino_total_testados]
cores = ["#1f77b4", "#ff69b4"]  # Azul para masculino, Rosa para feminino

# Criar gráfico de Donut
fig, ax = plt.subplots(figsize=(8, 6))
wedges, texts, autotexts = ax.pie(
    total_testados,
    labels=labels,
    autopct=lambda p: f"{p:.1f}%" if p > 0 else "",
    startangle=90,
    wedgeprops=dict(width=0.4, edgecolor="w"),
    colors=cores,  # Aplicar cores
)

# Títulos e Estilo
plt.setp(autotexts, size=12, weight="bold", color="black")
ax.set_title("Distribuição de Testados por Gênero", fontsize=16, fontweight="bold")

# Adicionar informações de taxa de positividade ao centro
info_text = (
    f"Masculino: {masculino_positividade:.1f}%\n"
    f"Feminino: {feminino_positividade:.1f}%"
)
ax.text(0, 0, info_text, ha="center", va="center", fontsize=12, weight="bold", color="black")

# Ajustar layout e mostrar
plt.tight_layout()
plt.show()


In [ ]:

# Dados de exemplo
dados = {
    "mes": ["Jan", "Fev", "Mar", "Abr", "Mai", "Jun"],
    "total_casos": [1000, 1200, 1500, 1300, 1700, 1800],
    "total_obitos": [50, 60, 70, 65, 85, 90],
}

df = pd.DataFrame(dados)

# Calcular percentual de óbitos
df["percentual_obitos"] = (df["total_obitos"] / df["total_casos"]) * 100

# Criar o gráfico
fig, ax1 = plt.subplots(figsize=(10, 6))

# Gráfico de barras para total de casos
ax1.bar(df["mes"], df["total_casos"], color="steelblue", label="Total de Casos")
ax1.set_xlabel("Mês", fontsize=12)
ax1.set_ylabel("Total de Casos", fontsize=12)
ax1.tick_params(axis="y")
ax1.legend(loc="upper left", fontsize=10)

# Remover linhas de grade do eixo Y principal (barras)
ax1.grid(axis='y', visible=False)

# Gráfico de linha para percentual de óbitos
ax2 = ax1.twinx()  # Segundo eixo y
ax2.plot(df["mes"], df["percentual_obitos"], color="gray", marker="o", label="% de Óbitos", linewidth=2)
ax2.set_ylabel("Percentual de Óbitos (%)", fontsize=12, color="gray")
ax2.tick_params(axis="y", labelcolor="gray")
ax2.legend(loc="upper right", fontsize=10)
ax2.grid(axis='y', visible=False)

# Adicionar rótulos nos pontos da linha
for i, valor in enumerate(df["percentual_obitos"]):
    ax2.text(i, valor, f"{valor:.1f}%", color="black", fontsize=10, ha="center", va="bottom")

# Ajustar limites do eixo Y para garantir espaço para os rótulos
ax2.set_ylim(0, df["percentual_obitos"].max() + 10)

# Melhorar a exibição dos rótulos do eixo X
ax1.set_xticks(range(len(df["mes"])))
ax1.set_xticklabels(df["mes"], fontsize=10)

# Título
plt.title("Total de Casos e Percentual de Óbitos por Mês", fontsize=16, fontweight="bold")

# Ajustar layout e exibir
plt.tight_layout()
plt.show()
